# GPT QA Rewriting

In [1]:
import time
import json
import pandas as pd
import numpy as np

from typing import Union
from tqdm import tqdm

import openai
from openai import AsyncOpenAI
from openai import OpenAI

import asyncio
import nest_asyncio

## Import Data

In [2]:
with open('../../data/a_ha_original.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data).transpose()
df.head()

,urls,title,question,answer
0,https://www.a-ha.io/questions/4cb726a43f6fea7f...,상법에서 정한 기한을 넘어선 주주총회의사록 공증 진행시 효력이 있는지요?,"안녕하세요,주주총회를 진행한 이후 일정 기간안에 법무사를 통한 공증을 진행해야 하는...",[안녕하세요. 이성재 변호사입니다.주주총회 결의사항 중 등기사항이 없다면 공증을 받...
1,https://www.a-ha.io/questions/4a8035d424b93c9e...,형사사법포털에는 기소유예가 떴는데 아직 어떠한 연락도 안와서요 혹시 왜그런건가요?,점유이탈횡령죄로 조사받고 얼마전 2일 기소유예 처분을 받았습니다. 근데 연락 받아서...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...
2,https://www.a-ha.io/questions/496d83cb9dfc2408...,자본금 10억 미만의 법인회사에서 이사가 1명일 때 발생하는 문제점,"안녕하세요,자본금 10억 미만의 회사지만 외부에서 투자를 많이 받고 있습니다.자본금...","[10억 미만의 회사이고 이사가 과반주주권자라면,주주총회 등을 적법하게 통지하고 개..."
3,https://www.a-ha.io/questions/4860dd4377505d30...,모든 계약은 법적으로 15일이내 해지가 가능한지요~?,한 회사(판매 사업)에 지사 계약을 일주일전에 했는데 할수없는상황이 되서 해지를 하...,[모든 계약에 대해서는 15일 만에 해지가 가능하다고 하는 규정은 따로 없습니다해당...
4,https://www.a-ha.io/questions/4afe128df4bd0726...,항소심 재판은 어떻게 진행되는건가요?,아는 지인이 항소후에 항소심 재판이 열리는데 어떻게 진행되는지 궁금합니다. 혹시 변...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...


In [3]:
df.rename(columns={'title': '제목'}, inplace=True)
df.rename(columns={'question': '본문'}, inplace=True)
df.rename(columns={'answer': '대답'}, inplace=True)
df.head()

,urls,제목,본문,대답
0,https://www.a-ha.io/questions/4cb726a43f6fea7f...,상법에서 정한 기한을 넘어선 주주총회의사록 공증 진행시 효력이 있는지요?,"안녕하세요,주주총회를 진행한 이후 일정 기간안에 법무사를 통한 공증을 진행해야 하는...",[안녕하세요. 이성재 변호사입니다.주주총회 결의사항 중 등기사항이 없다면 공증을 받...
1,https://www.a-ha.io/questions/4a8035d424b93c9e...,형사사법포털에는 기소유예가 떴는데 아직 어떠한 연락도 안와서요 혹시 왜그런건가요?,점유이탈횡령죄로 조사받고 얼마전 2일 기소유예 처분을 받았습니다. 근데 연락 받아서...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...
2,https://www.a-ha.io/questions/496d83cb9dfc2408...,자본금 10억 미만의 법인회사에서 이사가 1명일 때 발생하는 문제점,"안녕하세요,자본금 10억 미만의 회사지만 외부에서 투자를 많이 받고 있습니다.자본금...","[10억 미만의 회사이고 이사가 과반주주권자라면,주주총회 등을 적법하게 통지하고 개..."
3,https://www.a-ha.io/questions/4860dd4377505d30...,모든 계약은 법적으로 15일이내 해지가 가능한지요~?,한 회사(판매 사업)에 지사 계약을 일주일전에 했는데 할수없는상황이 되서 해지를 하...,[모든 계약에 대해서는 15일 만에 해지가 가능하다고 하는 규정은 따로 없습니다해당...
4,https://www.a-ha.io/questions/4afe128df4bd0726...,항소심 재판은 어떻게 진행되는건가요?,아는 지인이 항소후에 항소심 재판이 열리는데 어떻게 진행되는지 궁금합니다. 혹시 변...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...


## GPT API

In [4]:
nest_asyncio.apply()

client = AsyncOpenAI(
    api_key = '' 
)

## Prompt Generation

In [5]:
with open('../prompt/AHA_Prompt.txt', 'r') as f:
    prompt_file = f.read()

In [ ]:
# gpt-4

async def gpt_generate(title, body, original_answer):
    prompt = prompt_file.format(title, body, original_answer)

    completion = await client.chat.completions.create(
                    model = 'gpt-4', 
                    messages = [
                        {"role": "user",
                         "content": [
                            {"type": "text", "text": f"{prompt}"},
                            ]}], 
                    max_tokens = 300,
                    n = 1,
                    temperature = 0)

    return completion.choices[0].message.content

In [17]:
# gpt-4-turbo

async def gpt_generate(title, body, original_answer):
    prompt = prompt_file.format(title, body, original_answer)

    completion = await client.chat.completions.create(
                    model = 'gpt-4-turbo',
                    messages = [
                        {"role": "user",
                         "content": [
                            {"type": "text", "text": f"{prompt}"},
                            ]}], 
                    max_tokens = 300,
                    n = 1,
                    temperature = 0)

    return completion.choices[0].message.content

In [16]:
# gpt-3.5-turbo

def gpt_generate(title, body, original_answer):
    prompt = prompt_file.format(title, body, original_answer)
    
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"{prompt}"}
    ]
    )

    return completion.choices[0].message.content

In [20]:
# gpt-3.5-turbo-instruct

def gpt_generate(title, body, original_answer):
    prompt = prompt_file.format(title, body, original_answer)
    
    completion = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt
    )

    return completion.choices[0]

In [26]:
df.tail()

,urls,제목,본문,대답
111330,https://www.a-ha.io/questions/4a32d8fe397217f5...,전세보증금 선순위에 대해서 질문하고싶습니다,현재 1억8천짜리 전세에 살고 있는데요 지금 매매가는 2억3천정도 입니다. 다행히 ...,[안녕하십니까월드클래스 법률사무소의 이승환 대표변호사입니다전입신고와 확정일자를 받았...
111331,https://www.a-ha.io/questions/4e414d01a0773f29...,공증이란 것이 법적으로 효력이 있는 것인가요?,"친한 친구가 공증을 받고 1300만원을 다른 사람에게 빌려줬다던데, 공증이 법적 효...",[안녕하십니까월드클래스 법률사무소의 이승환 변호사입니다.공증과 관련하여 사서증서 인...
111332,https://www.a-ha.io/questions/413289513d835fb8...,경매로 인한 채권회수에 관한 내용입니다. 답변 부탁드립니다.,전세로 살던집이 경매로 매각이 되어 버렸습니다. 임차인은 후순위 채권자로 경매로인한...,[법무법인 에스엔의 홍경열 변호사입니다임차인은 전입신고와 확정일자를 갖춘경우 그 일...
111333,https://www.a-ha.io/questions/44df3de51a8f6e3d...,등기의 공신력 문의 드립니다.,우리나라는 부동산등기에 대한 공신력을 인정 받지 못하고 있습니다. 국민들은 부동산매...,[법무법인 에스엔의 홍경열 변호사입니다질문자께서 말씀하신바와 같이 부동산등기에는 공...
111334,https://www.a-ha.io/questions/4a18042f09713d19...,부동산등기에 관한 법률 질문 입니다,부동산 등기부에는 등기 선순위 저당권이 없어 최우선우 순위로 생각하고 임대계약을 ...,[안녕하십니까월드클래스 법률사무소의 이승환변호사입니다.주택임대차보호법에 따르는 경우...


In [18]:
result_list4_tail = []

for i in df.tail().iterrows():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(gpt_generate(i[1]['제목'], i[1]['본문'], i[1]['대답']))
    result_list4_tail.append(result)

In [101]:
# gpt-4
result_list2

['질문: 전세에 살고 있는데 집주인이 전세금을 돌려주지 못할 경우, 법률적으로 제가 1순위가 되는지와 돌려받지 못할 시 법적 처리 방법은 어떻게 되는지 알고 싶습니다.\n답변: 전입신고와 확정일자를 받았고, 그 이전에 근저당을 처리했다면, 법률적으로 보증금 반환에 대한 1순위가 됩니다. 만약 보증금을 돌려받지 못한다면, 법원에 보증금 반환 소송을 진행하여 확정판결을 받은 후 부동산에 대한 강제경매를 진행할 수 있습니다. 그리고 전부 돌려받지 못하는 경우에는 임대인의 다른 부동산, 동산 등에 대해서도 강제집행을 할 수 있습니다. 다만, 이 과정에서 미리 가압류, 가처분 등이 필요할 수 있습니다.',
 '질문: 친구가 1300만원을 빌려줄 때 공증을 받았는데, 이 공증이 법적 효력이 있는지, 그리고 그 효력이 어느 정도인지 알고 싶습니다.\n답변: 공증은 법적 효력이 있습니다. 공증은 사서증서의 인증을 받는 경우, 작성명의인의 의사에 따라 작성되었다는 것을 공증인이 확인하는 것입니다. 특히, 금전소비대차(돈을 빌려주는 것)를 할 때 강제집행 인낙의 표시가 있는 공정증서를 작성(공증인이 작성)하면, 법원의 확정판결 없이 바로 강제집행을 할 수 있습니다.',
 '질문: 전세로 살던 집이 경매로 매각되어 일부만 채권을 돌려받았을 경우, 임대인에 대한 채권회수를 위한 별도의 소를 진행할 수 있는지, 그리고 그 방법과 절차는 무엇인지 알고 싶습니다.\n답변: 임차인은 전입신고와 확정일자를 갖춘 경우 저당권자와 유사한 위치를 가지므로 반드시 후순위권자라고 할 수는 없습니다. 그러나 확정일자를 받아두지 못하였거나 선순위 저당권자들보다 순위가 낮아 경매절차에서 임차보증금을 전액 회수하지 못한 경우에는 남은 채권에 대하여 청구할 수 있고, 소송이나 압류 등 후속절차를 진행할 수 있습니다. 이 경우 임대인의 다른 부동산이나 동산 등에 대해 먼저 보전처분(가압류, 가처분)을 하고, 본안 소송을 제기한 후 확정이 된다면 집행절차에 착수하면 됩니다. 그러나 임대인에게 별다른 재산이나 

In [19]:
# gpt-4-turbo
result_list4_tail

['새로운 질문 및 답변:\n  질문: 전세보증금 반환 문제에서 전입신고와 확정일자를 받은 후 근저당 처리가 완료된 경우, 법률적으로 선순위가 되는지와 전세금을 돌려받지 못할 시 법적 처리 방법은 무엇인가요?\n  답변: 전입신고와 확정일자를 받은 상태에서 근저당이 없다면 법률적으로 선순위가 맞습니다. 만약 집주인이 전세금을 반환하지 못할 경우, 법원에 보증금 반환 소송을 제기하여 확정 판결을 받은 후 부동산에 대한 강제경매를 진행할 수 있습니다. 이 과정에서 전세금을 전부 돌려받지 못하는 경우, 임대인의 다른 부동산이나 동산에 대해서도 강제집행을 할 수 있으며, 이를 위해 가압류나 가처분 등의 조',
 '새로운 질문 및 답변:\n  질문: 친구가 1300만원을 빌려줄 때 공증을 받았는데, 이 공증이 법적으로 어떤 효력을 가지며, 어느 정도까지 효력이 있는지 궁금합니다.\n  답변: 공증은 문서가 진정으로 작성된 것임을 공증인이 확인하는 절차입니다. 특히 금전을 빌려주는 경우, 공정증서에 강제집행 인낙의 표시가 있으면, 법원의 확정판결 없이도 바로 강제집행을 할 수 있는 법적 효력을 가집니다. 이는 채무자가 채무 이행을 하지 않을 경우, 채권자가 더 신속하게 자신의 권리를 실현할 수 있도록 돕습니다.',
 '새로운 질문 및 답변:\n  질문: 전세로 살던 집이 경매로 매각된 후, 임차인이 후순위 채권자로서 전세보증금을 일부만 회수할 수 있었을 때, 나머지 보증금을 임대인에게서 회수할 수 있는 방법과 절차는 무엇인가요?\n  답변: 임차인이 전입신고와 확정일자를 갖추었다면, 저당권자와 유사한 위치를 가질 수 있으며, 이 경우 확정일자를 받지 못했거나 선순위 저당권자보다 순위가 낮아 전액을 회수하지 못했다면, 남은 채권에 대해 임대인에게 청구할 수 있습니다. 이를 위해 먼저 보전처분(가압류, 가처분)을 신청하고, 본안 소송을 제기한',
 '새로운 질문 및 답변:\n  질문: 우리나라에서 부동산등기부의 공신력이 인정되지 않는 경우, 부동산 매매 시 발생하는 문제와 

In [18]:
# gpt-3.5-turbo
result_list2_tail

[ChatCompletionMessage(content='질문: 만약 전세보증금을 돌려받지 못한 경우, 법적으로 1순위가 되는 것은 무엇인가요? 그리고 법적 처리는 어떻게 이루어지나요?\n답변: 보증금을 돌려받지 못한 경우, 법적으로 1순위가 됩니다. 이 경우 법원에 보증금반환 소송을 제기하여 확정판결을 받은 후 부동산에 대한 강제경매를 진행할 수 있습니다. 또한, 전부 돌려받지 못하는 경우 임대인의 다른 부동산, 동산 등에 대해서도 강제집행을 할 수 있습니다(다만 미리 가압류, 가처분등이 필요할 수 있습니다).', role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='질문:  친구가 공증으로 1300만원을 다른 사람에게 빌려줬다는데, 공증이 법적으로 효력이 있는 것인가요? 효력이 있다면 어느 정도까지 효력이 있는지 알려주세요.\n답변: 공증은 사서증서 인증을 받는 경우 진정한 작성명의인의 의사에 따라 작성된 것을 공증인이 확인하는 것입니다. 금전소비대차를 함에 있어 강제집행 인낙의 표시가 있는 공정증서를 작성한 경우, 법원의 확정판결 없이도 바로 강제집행을 할 수 있습니다.', role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='질문: 경매로 인한 채권회수시 임대인이 전세보증금을 회수하기 어려운 경우, 어떻게 채권을 확보하고 회수할 수 있을까요?\n답변: 임대인이 경매로 인한 채권회수를 위해 전세보증금을 일부만 받은 경우, 어떻게 채권을 확보하고 회수할 수 있는지에 대해 알려드리겠습니다. 임대인은 소송이나 압류 등 후속 절차를 통해 채권을 확보할 수 있습니다. 또한, 임대인의 다른 부동산이나 동산에 대해 보전처분(가압류, 가처분)을 한 뒤 본안 소송을 제기하여 확정이 된다면 집행절차에 착수할 수 있습니다. 따라서 임대인은 확정일자를 받아두고 부동산 시세

In [22]:
# gpt-3.5-turbo-instruct
result_list3_tail

[CompletionChoice(finish_reason='length', index=0, logprobs=None, text='\n 질문: 전입신고와 확정일자를'),
 CompletionChoice(finish_reason='length', index=0, logprobs=None, text='\n\n 질문: 친구가 공증을 받고'),
 CompletionChoice(finish_reason='length', index=0, logprobs=None, text='\n질문: 임대인보다 후순위'),
 CompletionChoice(finish_reason='length', index=0, logprobs=None, text='\n질문: 현재 부동산 매매에서 일'),
 CompletionChoice(finish_reason='length', index=0, logprobs=None, text='\n  질문: 임대인이 은행에서')]

In [19]:
print('재목: ', df['제목'].apply(len).mean())
print('본문: ', df['본문'].apply(len).mean())
print('대답: ', df['대답'].apply(len).mean())

재목:  28.888983697848836
본문:  227.26466070867204
대답:  2.220011676471909


----